## Try to get all hydrogens around metal atoms about a conventional cell

In [1]:
from pymatgen.core.structure import Structure
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from collections import defaultdict

## Count the amount of a specie in a cell

In [2]:
pmg_struct = Structure.from_file("./CaH6.cif")
pmg_struct.composition.to_data_dict
# {
#   'reduced_cell_composition': Comp: Li2 Mg1 H16,
#   'unit_cell_composition': defaultdict(float,
#              {'Li': 16.0, 'Mg': 8.0, 'H': 128.0}),
#   'reduced_cell_formula': 'Li2MgH16',
#   'elements': ['Li', 'Mg', 'H'],
#   'nelements': 3
# }
pmg_struct.composition.num_atoms            # get amounts of all atoms
pmg_struct.composition.alphabetical_formula # 'H128 Li16 Mg8'
pmg_struct.composition.anonymized_formula   # 'AB2C16'
pmg_struct.composition.iupac_formula        # 'Li16 Mg8 H128'
pmg_struct.composition.chemical_system      # 'H-Li-Mg'
pmg_struct.composition.elements             # [Element Li, Element Mg, Element H]
pmg_struct.composition.special_formulas     #{
                                            #     'LiO': 'Li2O2',
                                            #     'NaO': 'Na2O2',
                                            #     'KO': 'K2O2',
                                            #     'HO': 'H2O2',
                                            #     'CsO': 'Cs2O2',
                                            #     'RbO': 'Rb2O2',
                                            #     'O': 'O2',
                                            #     'N': 'N2',
                                            #     'F': 'F2',
                                            #     'Cl': 'Cl2',
                                            #     'H': 'H2'
                                            # }
pmg_struct.composition.get_el_amt_dict()                    # defaultdict(float, {'Li': 16.0, 'Mg': 8.0, 'H': 128.0})
pmg_struct.composition.get_integer_formula_and_factor()     # ('Li2MgH16', 8.0)
pmg_struct.composition.get_reduced_composition_and_factor() # (Comp: Li2 Mg1 H16, 8)


e:\conda\envs\cage\lib\site-packages\pymatgen\io\cif.py:1229: UserWarning: Issues encountered while parsing CIF: Skipping relative stoichiometry check because CIF does not contain formula keys.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))


(Composition('Ca1 H6'), 2)

## **一个小工具** 输入一个结构，输出其约化化学式，空间群对称性，文件名

In [3]:
import sys
from pathlib import Path

from pymatgen.core.structure import Structure
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer


struct = Structure.from_file("../test/POSCAR")
name = struct.composition.get_integer_formula_and_factor()[0]
spg = SpacegroupAnalyzer(struct)
num = spg.get_space_group_number()
spg_symbol = spg.get_space_group_symbol()
print(f"{num}-{spg_symbol}-{name}")
struct

229-Im-3m-CaH6


Structure Summary
Lattice
    abc : 2.9417705172008235 2.9417705172008235 2.9417705172008235
 angles : 109.47122063449069 109.47122063449069 109.47122063449069
 volume : 19.59767190050911
      A : -1.698432 1.698432 1.698432
      B : 1.698432 -1.698432 1.698432
      C : 1.698432 1.698432 -1.698432
    pbc : True True True
PeriodicSite: Ca (0.0, 0.0, 0.0) [0.0, 0.0, 0.0]
PeriodicSite: H (0.8492, 1.11e-16, 1.698) [0.5, 0.75, 0.25]
PeriodicSite: H (0.8492, 1.698, 0.0) [0.5, 0.25, 0.75]
PeriodicSite: H (1.698, 0.8492, 0.0) [0.25, 0.5, 0.75]
PeriodicSite: H (1.11e-16, 0.8492, 1.698) [0.75, 0.5, 0.25]
PeriodicSite: H (1.11e-16, 1.698, 0.8492) [0.75, 0.25, 0.5]
PeriodicSite: H (1.698, 1.11e-16, 0.8492) [0.25, 0.75, 0.5]

### 计算共轭类

In [4]:
from pymatgen.core.structure import Molecule
from pymatgen.symmetry.analyzer import PointGroupAnalyzer

water_coords = [(0.0, 0.0, 0.0), (0.757, 0.586, 0.0), (-0.757, 0.586, 0.0)]

water_species = ["H", "H", "O"]

# 创建水分子对象
water_molecule = Molecule(water_species, water_coords)
pg = PointGroupAnalyzer(water_molecule, matrix_tolerance=0.001)
pgops = pg.get_symmetry_operations()
print(pgops)
print(pg.get_pointgroup())
water_molecule

[SymmOp(affine_matrix=array([[ 1., -0., -0.,  0.],
       [-0.,  1., -0.,  0.],
       [-0., -0., -1.,  0.],
       [ 0.,  0.,  0.,  1.]])), SymmOp(affine_matrix=array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]]))]
Cs


Molecule Summary
Site: H (0.0000, 0.0000, 0.0000)
Site: H (0.7570, 0.5860, 0.0000)
Site: O (-0.7570, 0.5860, 0.0000)

In [5]:
from numpy import array
from typing import List
from pymatgen.symmetry.analyzer import SymmOp
import numpy as np

def is_in_matrix(m:array,ms:List[array]):
    # 判断矩阵m是否在矩阵组成的列表ms中
    flag = True
    for _ in ms:
        if np.allclose(m, _):
            flag = True
            break
    else:
        flag = False
    return flag

def kill_duplicated_element(ms:array):
    # 在一组 3*3的矩阵中，找到重复的并删除
    new_ms = []
    for m in ms:
        if not new_ms:
            new_ms.append(m)
        else:
            if not is_in_matrix(m=m, ms=new_ms):
                new_ms.append(m)
    return new_ms

def is_equal_for_cless(cl1:List[array], cl2:List[array]):
    # 判断两个类是否相同
    flag = True
    for m1 in cl1:
        if is_in_matrix(m1, cl2):
            flag = True
            break
    else:
        flag = False
    return flag

def is_in_clesses(cl1:List[array], clesses:List[List[array]]):
    flag = True
    for cl2 in clesses:
        if is_equal_for_cless(cl1, cl2):
            flag = True
            break
    else:
        flag = False
    return flag

def kill_duplicated_clesses(clesses:List[List[array]]):
    new_clesses = []
    for cl in clesses: # cl此时是一个类，里面有很多个3*3的矩阵
        if not new_clesses:
            new_clesses.append(cl)
        else:
            if not is_in_clesses(cl1=cl, clesses=new_clesses):
                new_clesses.append(cl)
    return new_clesses


def find_cless_ops(cless:List[array]):
    tmplist = []
    for rot in cless:
        for op in pgops:
            if np.allclose(op.rotation_matrix, rot):
                tmplist.append(op)
                break
    return tmplist

def get_rot_name(rot):
    # print(rot)
    typename = ''
    det = np.linalg.det(rot) 
    if det > 0: # det(rot)=1: rotation; 
        trc = np.trace(rot)
        theta = np.degrees(np.arccos((trc-1)/2))
        # print(trc); print(theta)
        if np.allclose(theta, 0.0):
            typename = "identity"
        else:
            typename = "rotation"+str(int(theta))
    else: # det(rot)=-1: rotoinversion
        trc = -np.trace(rot)
        theta = np.degrees(np.arccos((trc-1)/2))
        # print(trc); print(theta)
        if np.allclose(theta, 0.0):
            typename = "inversion"
        else:
            typename = "rotoinversion"+str(int(theta))

    return typename

def get_cless_name(cless:List[SymmOp]):
    namelist = []
    for op in cless:
        name = get_rot_name(op.rotation_matrix)
        namelist.append(name)
    
    if len(set(namelist)) == 1:
        return namelist[0]+ "_" + str(len(namelist))
    else:
        raise ValueError("Error: namelist={}".format(namelist))

def classify_ops(pgops:List[SymmOp]):
    clesseslist: List[array] = []
    for idx, op in enumerate(pgops):
        # print(op.as_xyz_str())
        bm = [np.dot(np.dot(np.linalg.inv(x.rotation_matrix), op.rotation_matrix), x.rotation_matrix) for x in pgops]
        # print(bm)
        cless = kill_duplicated_element(bm)
        # print(cless)
        # for m in cless:
        #     print(detailed_info(m))
        # break
        clesseslist.append(cless)

    clesseslist = kill_duplicated_clesses(clesseslist)

    clessesdict = defaultdict(list)
    for idx, cless in enumerate(clesseslist):
        tmp:List[SymmOp] = find_cless_ops(cless)
        clessname:str = get_cless_name(tmp)
        clessesdict[f'{clessname}'] = tmp

    clessesdict:dict[List[SymmOp]] = dict(clessesdict)
    return clessesdict

print(pgops)

new_pgops:dict[List[SymmOp]] = classify_ops(pgops)
print(new_pgops.keys())

[SymmOp(affine_matrix=array([[ 1., -0., -0.,  0.],
       [-0.,  1., -0.,  0.],
       [-0., -0., -1.,  0.],
       [ 0.,  0.,  0.,  1.]])), SymmOp(affine_matrix=array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]]))]
dict_keys(['rotoinversion180_1', 'identity_1'])


In [6]:
def generate_mapping_matrix(mapping):
    """
    通过给定的映射关系生成矩阵 A

    Args:
        mapping: 一个字典，表示映射关系，键为第一列的元素，值为第二列的元素

    Returns:
        矩阵 A
    """
    # 确定矩阵 A 的大小
    size = len(mapping)
    A = [[0] * size for _ in range(size)]

    # 遍历映射关系，填充矩阵 A
    for idx, key in enumerate(mapping):
        value = mapping[key]
        A[idx][value] = 1
    # print(mapping)
    return A


def get_atoms_mapping(op:SymmOp, struct:Structure):

    from pymatgen.core.sites import PeriodicSite
    
    mapping = {}
    for idx1, site1 in enumerate(struct):
        newcoords = op.operate(site1.frac_coords)
        newsite = PeriodicSite(species=site1.species, coords=newcoords, to_unit_cell=True, lattice=struct.lattice)
        for idx2, site2 in enumerate(struct):
            if np.allclose(newsite.coords, site2.coords, rtol=1e-4):  # pymatgen 源代码检查了元素是否相等，坐标是否相等，性质是否相等 self.species == other.species and np.allclose(self.coords, other.coords, atol=Site.position_atol) and self.properties == other.properties
                # print("{} {}-> {} {}".format(idx1, site1.frac_coords, idx2, newsite.frac_coords))
                # print("{} {}-> {} {}".format(idx1, site1, idx2, newsite))
                mapping[idx1] = idx2
    return mapping

# for name in ['identity_1', 'rotation180_3', 'rotation180_6', 'rotation120_8', 'rotation90_6', 'inversion_1', 'rotoinversion180_3', 'rotoinversion180_6', 'rotoinversion120_8', 'rotoinversion90_6']:
#     print(get_atoms_mapping(new_pgops[name][0], struct=struct))

def get_conjugate_character(ops:List[SymmOp], struct):

    trcs    = []
    for op in ops:
        mapping = get_atoms_mapping(op, struct)
        A = generate_mapping_matrix(mapping)
        trc = np.trace(A)
        trcs.append(trc)
    if len(set(trcs)) == 1:
        return trc

# for name in ['identity_1', 'rotation180_3', 'rotation180_6', 'rotation120_8', 'rotation90_6', 'inversion_1', 'rotoinversion180_3', 'rotoinversion180_6', 'rotoinversion120_8', 'rotoinversion90_6']:
#     print(get_conjugate_character(new_pgops[name], struct=struct))

def get_atomic_character(new_pgops, struct):
    
    atomic_charater = {}
    for name, ops in new_pgops.items():
        print("---- check cless {} ----".format(name))
        trc = get_conjugate_character(ops, struct)
        atomic_charater[name] = trc
    return atomic_charater

conjugate_charater = get_atomic_character(new_pgops, struct)
# for name in ['identity_1', 'rotation180_3', 'rotation180_6', 'rotation120_8', 'rotation90_6', 'inversion_1', 'rotoinversion180_3', 'rotoinversion180_6', 'rotoinversion120_8', 'rotoinversion90_6']:
# names = ['identity_1','rotation90_6', 'rotation180_3', 'rotation120_8', 'rotation180_6',  'inversion_1', 'rotoinversion90_6', 'rotoinversion180_3',  'rotoinversion120_8', 'rotoinversion180_6', ]

# mult  = [int(name.split('_')[-1]) for name in ['identity_1','rotation90_6', 'rotation180_3', 'rotation120_8', 'rotation180_6',  'inversion_1', 'rotoinversion90_6', 'rotoinversion180_3',  'rotoinversion120_8', 'rotoinversion180_6', ]]
# print(mult)

# A1g = [1,1,1,1,1, 1,1,1,1,1]
# A1u = [1,1,1,1,1, -1,-1,-1,-1,-1]
# red_rep = list(conjugate_charater.values())
# print(np.array(red_rep)-1)

# from numpy import int32
# x=np.sum(np.array(mult)*np.array(A1u)*(np.array(red_rep)-1))/np.sum(mult)
# x

---- check cless rotoinversion180_1 ----


IndexError: list assignment index out of range

In [ ]:
x = np.array([1,2,3])
n = np.array([0,1,3])
i = np.array([1,2,4])
x*n*i

array([ 0,  4, 36])

## Voronoi模块的使用

In [ ]:
from pymatgen.core.structure import Structure
from pymatgen.analysis.chemenv.coordination_environments.structure_environments \
    import ChemicalEnvironments, StructureEnvironments, DetailedVoronoiContainer
from pymatgen.analysis.chemenv.coordination_environments.chemenv_strategies \
    import SimplestChemenvStrategy

structure = Structure.from_file("./CaH6.cif")
vc = DetailedVoronoiContainer(structure)
# se = StructureEnvironments(voronoi_container)
# sc = SimplestChemenvStrategy()
# sc.get_site_neighbors(structure)

e:\conda\envs\cage\lib\site-packages\pymatgen\io\cif.py:1229: UserWarning: Issues encountered while parsing CIF: Skipping relative stoichiometry check because CIF does not contain formula keys.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))


In [ ]:
from pymatgen.analysis.graphs import StructureGraph
from pymatgen.analysis.local_env import NearNeighbors

s = Structure.from_file("./La2Y6H46_ContributedBy_Mayuan.vasp")
# graph = StructureGraph.with_local_env_strategy(structure, strategy=nn)

In [ ]:
from pymatgen.analysis.local_env import VoronoiNN

vnn = VoronoiNN()
all_vr = vnn.get_all_voronoi_polyhedra(structure)
all_vr

[{1499: {'site': PeriodicNeighbor: H (-1.202, -2.404, -2.208e-16) [-0.25, -0.5, 0.0],
   'normal': array([-4.47213595e-01, -8.94427191e-01, -8.21518047e-17]),
   'solid_angle': 0.5235987755982987,
   'volume': 0.502250242311989,
   'face_dist': 1.343650452594652,
   'area': 1.1213859408348061,
   'n_verts': 3,
   'verts': [738, 737, 736],
   'adj_neighbors': [847, 978, 699]},
  847: {'site': PeriodicNeighbor: H (-3.865e-16, -2.404, 1.202) [0.0, -0.5, 0.25],
   'normal': array([-1.43834862e-16, -8.94427191e-01,  4.47213595e-01]),
   'solid_angle': 0.5235987755982989,
   'volume': 0.5022502423119891,
   'face_dist': 1.3436504525946518,
   'area': 1.1213859408348066,
   'n_verts': 3,
   'verts': [737, 1043, 738],
   'adj_neighbors': [1499, 1379, 1244]},
  1379: {'site': PeriodicNeighbor: H (1.202, -2.404, -7.359e-17) [0.25, -0.5, 0.0],
   'normal': array([ 4.47213595e-01, -8.94427191e-01, -2.73839349e-17]),
   'solid_angle': 0.5235987755982989,
   'volume': 0.5022502423119891,
   'face_di

In [ ]:
from pymatgen.analysis.local_env import VoronoiNN

vnn = VoronoiNN()
all_vr = vnn.get_all_voronoi_polyhedra(structure)
all_vr

[{1499: {'site': PeriodicNeighbor: H (-1.202, -2.404, -2.208e-16) [-0.25, -0.5, 0.0],
   'normal': array([-4.47213595e-01, -8.94427191e-01, -8.21518047e-17]),
   'solid_angle': 0.5235987755982987,
   'volume': 0.502250242311989,
   'face_dist': 1.343650452594652,
   'area': 1.1213859408348061,
   'n_verts': 3,
   'verts': [738, 737, 736],
   'adj_neighbors': [847, 978, 699]},
  847: {'site': PeriodicNeighbor: H (-3.865e-16, -2.404, 1.202) [0.0, -0.5, 0.25],
   'normal': array([-1.43834862e-16, -8.94427191e-01,  4.47213595e-01]),
   'solid_angle': 0.5235987755982989,
   'volume': 0.5022502423119891,
   'face_dist': 1.3436504525946518,
   'area': 1.1213859408348066,
   'n_verts': 3,
   'verts': [737, 1043, 738],
   'adj_neighbors': [1499, 1379, 1244]},
  1379: {'site': PeriodicNeighbor: H (1.202, -2.404, -7.359e-17) [0.25, -0.5, 0.0],
   'normal': array([ 4.47213595e-01, -8.94427191e-01, -2.73839349e-17]),
   'solid_angle': 0.5235987755982989,
   'volume': 0.5022502423119891,
   'face_di

# 获得一个晶体晶胞的原子数

In [ ]:
from pymatgen.core.structure import Structure
pmg_struct = Structure.from_file("./CaH6.cif")
pmg_struct.density
pmg_struct.num_sites

14

# 替换元素

In [ ]:
from pymatgen.core.structure import Structure
from pymatgen.io.vasp import Poscar
# filenames1 = ["LaCeH8-200GPa-P4mmm", "LaCeH8-200GPa-Pmmn", "LaCeH18-200GPa-Amm2", 
#              "LaCeH20-200GPa-I41amd", "LaCeH20-200GPa-P-6m2", "LaCeH20-200GPa-P63mmc",
#             #  "LaCeH20-200GPa-R-3m",]
filenames2 = ["Y1Ce1H7-100GPa-Pmma", "YCeH5-100GPa-C2m", "YCeH8-100GPa-P4mmm", "YCeH18-200GPa-P-6m2", "YCeH20-400-P4mmm"]

for file in filenames2:
    s = Structure.from_file(file+".cif")
    s.replace_species({'Y':'Ce', 'Ce':'Sr'})
    formula=s.composition.reduced_formula.replace(' ', '')
    filename="f1-"+formula+'-'+'-'.join(file.split('-')[1:])
    print(filename)
    Poscar(s).write_file(filename=filename+".vasp")

FileNotFoundError: [Errno 2] No such file or directory: 'Y1Ce1H7-100GPa-Pmma.cif'